In [ ]:
# 손가락 나오는거 : 20

In [ ]:
##############손가락 검출 성공케이스#######################################

In [1]:
import cv2
import numpy as np

# 이미지를 로드합니다.
img = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC20.jpg")

In [2]:
def img_process(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (13, 13), 0)
    
    ret, dst = cv2.threshold(blur, 212, 240, cv2.THRESH_BINARY)
    
    # 구조 요소 생성
    kernel = np.ones((9, 9), np.uint8)  # 구조 요소 크기와 형태 지정 (조정 가능)
    
    # 팽창 연산 적용
    dilated_image = cv2.dilate(dst, kernel, iterations=1)

    
    # 윤곽선 찾기
    contours, _ = cv2.findContours(dilated_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 새로운 캔버스 생성
    canvas = np.zeros_like(img)

    # 면적이 원본 이미지의 10% 이상인 윤곽선 그리기
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 0.2* img.shape[0] * img.shape[1]:
            # 윤곽선 근사화
            epsilon = 0.03 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            cv2.drawContours(canvas, [approx], -1, (0, 255, 0), 2)
            
    # 원근 변환 적용
    src_pts = np.float32(approx)
    dst_pts = np.float32([[0, 0], [0, img.shape[0]], [img.shape[1], img.shape[0]],[img.shape[1], 0]])
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped_image = cv2.warpPerspective(img, matrix, (img.shape[1], img.shape[0])) #flags=cv2.INTER_CUBIC
    #sharp = cv2.filter2D(warped_image, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
    
    # 이미지 크기 조정
    resized_image = cv2.resize(warped_image, (400, 200))
    #resized_img = cv2.resize(sharp, (400, 200))
    
    #결과 폴더 저장
    cv2.imwrite("C:/Users/Yunji Noh/OneDrive/image_processing/results/final_image/BC20.jpg", resized_img )
    cv2.imwrite("C:/Users/Yunji Noh/OneDrive/image_processing/results/final_image(sharp(x))/BC20.jpg", resized_image)
    #cv2.imwrite("C:/Users/Yunji Noh/OneDrive/image_processing/project_mid/image_cubic/BC20_cubic.jpg", resized_image)
    
    # 결과 이미지 출력
    cv2.imshow("final Image", resized_img)
    cv2.imshow('Contours', canvas)
    cv2.imshow('dilated_image.jpg', dilated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
img_process(img)

In [ ]:
##############################실패 케이스############################################

In [ ]:
import cv2
import numpy as np

# 이미지를 로드합니다.
img = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC20.jpg")

# 이미지 전처리
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (11, 11), 0)

# Canny 엣지 검출을 수행합니다.
edges = cv2.Canny(blur, 80, 130)

# 팽창과 침식을 위한 커널 생성
kernel = np.ones((7,7), np.uint8)

# 닫기 연산 수행
closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

# 윤곽선 검출
contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 명함으로 추정되는 윤곽선을 그리기 위한 캔버스 생성
canvas = np.zeros_like(img)

# 윤곽선을 순회하며 명함으로 추정되는 윤곽선을 캔버스에 그립니다.
for contour in contours:
    area = cv2.contourArea(contour)
    if area > img.shape[0] * img.shape[1] * 0.1:  # 윤곽선 면적이 원본 이미지의 10% 이상인 경우
        cv2.drawContours(canvas, [contour], -1, (0, 0, 255), 2)

# 결과 이미지 출력
cv2.imshow("Detected Business Card", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
######################실패케이스2 -> 원근변환 실패#################################

In [ ]:
import cv2
import numpy as np

# 이미지를 로드합니다.
img = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC20.jpg")

In [ ]:
def img_process(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (11, 11), 0)
    
    # 엣지 검출
    thres1 = 85
    thres2 = 95
    edge = cv2.Canny(blur, thres1, thres2)
    
    # 닫기 연산을 위한 구조 요소 생성
    kernel = np.ones((3, 3), np.uint8)
    
    # 닫기 연산 수행
    closed = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)
    
    # 컨투어 검출
    contours, _ = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # warped 이미지들을 저장할 리스트 생성
    warped_images = []
    
    # 컨투어 그리기 및 원근 변환
    for contour in contours:
        # 컨투어의 면적 계산
        area = cv2.contourArea(contour)
        
        # 컨투어의 길이 계산
        length = cv2.arcLength(contour, True)
        
        # 면적이 너무 작거나 길이가 작은 컨투어 제외
        if area > 150 and length > 250:
            # 컨투어를 감싸는 최소한의 사각형 구하기
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            # 원근 변환을 위한 좌표 설정
            src_pts = box.astype(np.float32)
            dst_pts = np.array([[0, 300],[0, 0],[400, 0], [400, 300] ], dtype=np.float32)
            
            # 원근 변환 행렬 계산
            M = cv2.getPerspectiveTransform(src_pts, dst_pts)
            
            # 원근 변환 적용
            warped = cv2.warpPerspective(img, M, (400, 300))
            
            # 변환 결과를 리스트에 저장
            warped_images.append(warped)
            
    # 원하는 순서의 이미지 선택하여 출력
    if len(warped_images) > 1:
        desired_index = 2  # 원하는 순서의 이미지 인덱스 선택
        cv2.imshow('Warped Image', warped_images[desired_index])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    # 결과 출력
    cv2.imshow('Contours', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


In [ ]:
img_process(img)